## **Intro**
### In this exercise, we’ll be fine-tuning a Llama2 model for generating image generation prompts from a short concept .
---
\
### **Instructions**

1. The exercise is defined in high-level terms on purpose, there are many different ways to solve each step and we wish to see how do you approach these kind of problems without any artificial restrictions.

1. **Make sure you understand all the code you will use, try to choose the most elegent and optimized solution**.

1. **Document as much as possible, feel free to use markdown for better description, and maintain a clean code according to best Python and coding practices**.

1. **The exercise should be submitted as a Google Colab notebook with the output**.

1. Feel free to add a Notes.docx, where you can share any technical thoughts and considerations you might have had, that might be useful to understand your code.

1. Please feel free to contact us if you have any questions.

1. Good Luck!!

---

In [ ]:
!pip install -q accelerate peft==0.4.0 bitsandbytes==0.40.0 transformers trl
!pip install langchain
!pip install langchain_openai
!pip install gdown

# Download the file using its Google Drive shareable link
!gdown --id 1oy19JQ9NBQ1KwcazW_TaMHj-gGbFd5cY -O /content/ds.json







## Step 1: Generate a dataset for llama2 fine-tuning
1. Use Mistral-7b or and other LLM that isn't Llama2
1. The dataset should be a pair of *concept* and *description* the an AI Image generator can use to generate an image, in a format that Llama2 can use for fine-tuning.
1. For example, concept: "A person is hiking", description: "A hiker moves up a sunlit path towards distant snow-capped mountains, surrounded by a lush forest. His determined stride and the serene natural backdrop form a picture of quiet adventure under a clear blue sky."  

In [ ]:
import ast
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import json
import pandas as pd

def get_dataset_from_llm():
    """
    Generate a dataset for LLAMA training using OpenAI's ChatOpenAI API.

    Returns:
    - result_json: Generated dataset in JSON format.
    """
    template = """
    You are an expert at generating datasets for llm training.
    You are an expert of llama2 text format.
    You need to create pairs of texts.
    Each pair will be a concept and description.
    The pair will describe a image that need to be generated.
    For example:
    concept: A person is hiking
    description: A hiker moves up a sunlit path towards distant snow-capped mountains, surrounded by a lush forest. His determined stride and serene natrual backdrop from a picture of a quite adventure under a clear blue sky.

    As you see the description should be very descriptive, with up to 35 words.

    Generate for me 10 of these pairs

    The result must be a valid json string. string should not start with json prefix.
    You should not add any additional information to the result.
    """
    llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.0, request_timeout=60, max_retries=2, api_key="####") #api_key="####" is ### for security
    prompt = ChatPromptTemplate.from_template(template=template)
    messages = prompt.format()
    response = llm.invoke(messages)
    try:
        result = response.content.replace("```{", "{").replace("```json", "").replace("```", "").replace("\n\t", "").replace("\n", "")
        result_json = json.loads(result)
        return result_json
    except Exception as e:
        raise e

def get_dataset_from_file():
    """
    Load dataset from a JSON file.

    Returns:
    - result_json: Loaded dataset in dictionary format.
    """
    f = open("ds.json", "r")
    result_json = f.read()
    return ast.literal_eval(result_json)

def get_and_divide_data():
    """
    Generate training and test datasets, and save them as CSV files.
    """
    result_json = get_dataset_from_file()  # Change to get_dataset_from_llm() for online generation
    df = pd.DataFrame.from_dict(result_json)
    test = df.sample(frac = 0.2)
    train = df.drop(test.index)
    train.to_csv("train.csv", index=False)
    test.to_csv("test.csv", index=False)


get_and_divide_data()


## Step 2: Split generated dataset into Training: 80%,  Test:20%

In [ ]:
# this also happens in get_and_divide_data()

## Step 3: Fine tune llama2 on Train set



In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding

def load_train_data(file_path):
    """
    Load training data from a CSV file.
    """
    return load_dataset("csv", data_files=file_path, split="train")

def train_model(num_train_epochs):
    """
    Train the model with the specified number of epochs.
    """
    model_name = "NousResearch/Llama-2-7b-chat-hf"
    new_model = "Llama-2-7b-chat-finetune"
    lora_r = 64
    lora_alpha = 16
    lora_dropout = 0.1
    use_4bit = True
    bnb_4bit_compute_dtype = "float16"
    bnb_4bit_quant_type = "nf4"
    use_nested_quant = False
    output_dir = "./results"
    fp16 = False
    bf16 = False
    per_device_train_batch_size = 4
    per_device_eval_batch_size = 4
    gradient_accumulation_steps = 1
    max_grad_norm = 0.3
    learning_rate = 2e-4
    weight_decay = 0.001
    optim = "paged_adamw_32bit"
    path = "/content/"

    # Load training data
    train_split = load_train_data("train.csv")

    # Load base model
    model = load_base_model(model_name, use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant)

    # Fine-tune model
    trainer ,tokenizer = fine_tune_model(model, train_split, lora_r, lora_alpha, lora_dropout, output_dir, num_train_epochs,
                                       fp16, bf16, per_device_train_batch_size, per_device_eval_batch_size,
                                       gradient_accumulation_steps, max_grad_norm, learning_rate, weight_decay,
                                       optim)

    # Save trained model
    trainer.model.save_pretrained(path)
    return trainer,tokenizer  # Return the trainer object


def load_base_model(model_name, use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant):
    """
    Load the base model with quantization configuration.
    """
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},output_hidden_states=True
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model




def fine_tune_model(model, train_dataset, lora_r, lora_alpha, lora_dropout, output_dir, num_train_epochs, fp16, bf16,
                    per_device_train_batch_size, per_device_eval_batch_size, gradient_accumulation_steps, max_grad_norm,
                    learning_rate, weight_decay, optim):
    """
    Fine-tune the model using the specified parameters.
    """
    model_name = "NousResearch/Llama-2-7b-chat-hf"
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
    )

    training_arguments = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        optim=optim,
        logging_steps=25,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        fp16=fp16,
        bf16=bf16,
        max_grad_norm=max_grad_norm,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="cosine",
        report_to="tensorboard"
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        peft_config=peft_config,
        dataset_text_field="concept",
        tokenizer=tokenizer,
        args=training_arguments,
    )
    trainer.train()

    return trainer,tokenizer


num_train_epochs = 1
train_split = load_train_data("train.csv")
trainer ,tokenizer = train_model(num_train_epochs)  # Get the trainer object


## Step 4: Evaluate the fine-tuned model on the Train set
1. For each entry in the Train set, compare the base model response and fine-tuned model response similarity to the expected results
1. Improvement ideas in *Notes.docx* would be welcome.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


def plot_similarity_distribution(train_split, trainer,tokenizer):
    similarities = []
    model_name = "NousResearch/Llama-2-7b-chat-hf"

    for entry in train_split:
        concept = entry["concept"]
        base_response = concept
        # Generate fine-tuned response
        inputs = tokenizer(concept, return_tensors="pt")
        fine_tuned_response = trainer.model.generate(**inputs)
        fine_tuned_response_str = tokenizer.decode(fine_tuned_response[0], skip_special_tokens=True)
        similarity = compare_responses(base_response, fine_tuned_response_str,trainer,tokenizer)
        similarities.append(similarity)
    plt.hist(similarities, bins=20, color='blue', alpha=0.7)
    plt.xlabel('Cosine Similarity')
    plt.ylabel('Data example')
    plt.title('Distribution of Cosine Similarity between Base and Fine-tuned Responses')
    plt.grid(True)
    plt.show()

def compare_responses(base_response, fine_tuned_response,trainer,tokenizer):
    base_embedding = generate_embedding(base_response,trainer,tokenizer)
    fine_tuned_embedding = generate_embedding(fine_tuned_response,trainer,tokenizer)
    return calculate_similarity(base_embedding, fine_tuned_embedding)

def generate_embedding(text, trainer, tokenizer):
  # i tried to use the orignial embedding by useing HuggingFaceEmbedding(model_name="NousResearch/Llama-2-7b-chat-hf")
  # but there werent enough avalibale RAM, so i used my model as the embedding function
  # in a real project i would have search for a better embedding function
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = trainer.model(**inputs)

    # Retrieve the hidden states from the model output
    hidden_states = outputs.hidden_states

    # Choose which hidden state to use for the embedding
    last_hidden_state = hidden_states[-1]

    # Use the mean of the last hidden state as the embedding
    embedding = last_hidden_state.mean(dim=1).squeeze().tolist()
    return embedding


def calculate_similarity(base_embedding, fine_tuned_embedding):
    return cosine_similarity([base_embedding], [fine_tuned_embedding])[0][0] # shape (1,1)


plot_similarity_distribution(train_split, trainer,tokenizer)


## Bonus Step: On "Llama-2-7b-chat-hf-sharded" or any other "chat fine-tuned" model, compare the first and last activation layers for 2 prompts
1. For each of the prompts: “How to build a snowman?” and “How to build a bomb?”
  1. Get the values vector of the first and the last activation layer on each forward-pass
  1. Calculate 2 average values vectors for the first and last activation layers (for all forward-passes)
1. Share your thoughts on the results for the 2 prompts and whether Alignment \ Guardrails have anything to do with it.
1. The main objective of this exercise is to see if you're comfortable with interacting with the models underlying layers, the actual similarty function and other "mathematical" considerations, are less important for this step.

In [ ]:
def extracting_layers(tokenizer):
    # Load the fine-tuned model and tokenizer
    model_name = "NousResearch/Llama-2-7b-chat-hf"
    use_4bit = True
    bnb_4bit_compute_dtype = "float16"
    bnb_4bit_quant_type = "nf4"
    use_nested_quant = False

    # Load base model
    model = load_base_model(model_name, use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant)

    # Define the prompts
    prompts = ["How to build a snowman?", "How to build a bomb?"]

    # Tokenize the prompts and convert them to input IDs
    input_ids_list = [tokenizer.encode(prompt, return_tensors="pt") for prompt in prompts]

    # List to store activation vectors for the first and last layers
    first_layer_activations_list = [[], []]
    last_layer_activations_list = [[], []]

    # Forward pass through the model for each prompt
    for i, input_ids in enumerate(input_ids_list):
        with torch.no_grad():
            outputs = model(input_ids)

            # Extract activations for the first and last layers
            first_layer_activations = outputs.hidden_states[0]  # First layer activations
            last_layer_activations = outputs.hidden_states[-1]  # Last layer activations

            # Select the first and last values vectors
            first_layer_activations_list[i].append(first_layer_activations)
            last_layer_activations_list[i].append(last_layer_activations)

            # # Generate response from the model
            # logits_flat = [logit for sublist in outputs.logits[0] for logit in sublist]
            # generated_response = tokenizer.decode(logits_flat, skip_special_tokens=False)
            # generated_responses.append(generated_response)


    # Plot activations for each prompt
    for i, (prompt, first_activations, last_activations) in enumerate(zip(prompts, first_layer_activations_list, last_layer_activations_list), start=1):
        num_inputs = len(first_activations)
        plt.figure(figsize=(10, 5 * num_inputs))

        for j, (first_layer_activation, last_layer_activation) in enumerate(zip(first_activations, last_activations), start=1):
            # Plot activations for the first layer
            plt.subplot(num_inputs, 2, 2 * j - 1)
            plt.plot(first_layer_activation.squeeze().tolist())
            plt.title(f'Prompt {i}, Input {j} - First Layer Activations')
            plt.xlabel('Token Position')
            plt.ylabel('Activation Value')

            # Plot activations for the last layer
            plt.subplot(num_inputs, 2, 2 * j)
            plt.plot(last_layer_activation.squeeze().tolist())
            plt.title(f'Prompt {i}, Input {j} - Last Layer Activations')
            plt.xlabel('Token Position')
            plt.ylabel('Activation Value')

        plt.tight_layout()
        plt.show()


extracting_layers(tokenizer)